<a href="https://colab.research.google.com/github/v3xlrm1nOwo1/Songs-Genre-Classification-by-Lyrics-With-BERT-And-LoRA/blob/main/Songs_Genre_Classification_by_Lyrics_With_BERT_And_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Music Genre Classification by Lyrics With BERT**

> BERT



## *Download Dataset and Lyary*

In [ ]:
!pip install kaggle

In [2]:
!mkdir ../root/.kaggle
!mv kaggle.json ../root/.kaggle

In [ ]:
!kaggle datasets download -d neisse/scrapped-lyrics-from-6-genres

In [ ]:
!ls

In [5]:
! mkdir music_genre_classification_by_lyrics

In [ ]:
! unzip scrapped-lyrics-from-6-genres.zip -d music_genre_classification_by_lyrics

In [ ]:
!ls

In [ ]:
!nvidia-smi

In [ ]:
!pip install git+https://github.com/thunlp/OpenDelta.git

In [ ]:
!pip install transformers[torch]

### import libraris



In [96]:
import transformers
import torch
import opendelta

import io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import shutil
import sys

from pylab import rcParams
from matplotlib import rc
from sklearn.model_selection import train_test_split
from textwrap import wrap
from collections import defaultdict
from accelerate import Accelerator

from torch import nn, optim
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification
import torch.nn.functional as F

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 10, 4

RANDOM_SEED = 666
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

LEARN_RATE = 3e-6
RANDOM_SEED = 666
MAX_LEN = 300
BATCH_SIZE = 8
EPOCHS = 3
CHECKPOINT= 'bert-base-cased'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Read and Preprocessing Data

In [ ]:
artists_df = pd.read_csv('music_genre_classification_by_lyrics/artists-data.csv')

artists_df.head()

In [ ]:
artists_df.shape

In [ ]:
len(artists_df.Genres.unique())

In [16]:
artists_df.rename(columns={'Genres':'Genre'}, inplace=True)

In [ ]:
lyrics_df = pd.read_csv('music_genre_classification_by_lyrics/lyrics-data.csv')

lyrics_df.head()

In [ ]:
lyrics_df.shape

In [ ]:
len(lyrics_df.Lyric.unique())

In [ ]:
artists_df = artists_df.drop_duplicates(subset = 'Link', keep ='first')

lyrics_df.shape

In [21]:
lyrics_df.rename(columns={'ALink':'Link'}, inplace=True)

In [22]:
lyrics_artists_df = pd.merge(lyrics_df ,artists_df, on='Link')

In [ ]:
lyrics_artists_df.head()

In [ ]:
lyrics_artists_df.shape

In [ ]:
lyrics_artists_df.columns

In [ ]:
lyrics_artists_df.language.unique()

In [ ]:
lyrics_artists_df.groupby('language')['Lyric'].count().sort_values(ascending=[0])[: 10]

In [ ]:
lyrics_artists_df = lyrics_artists_df[lyrics_artists_df['language'] == 'en']

lyrics_artists_df.shape

In [ ]:
lyrics_artists_df.head()

In [30]:
lyrics_artists_df = lyrics_artists_df[['Lyric', 'Genre', 'Artist', 'Popularity', 'SName']]
df = lyrics_artists_df[lyrics_artists_df.Lyric.notnull()]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
artists = lyrics_artists_df['Artist'].unique()
genres = lyrics_artists_df['Genre'].unique()

len(genres)

In [ ]:
df.groupby('Genre')['Lyric'].count().sort_values(ascending=[0])[: 20]

In [ ]:
'Rap; Hip Hop; Black Musi'.find('Rap')

In [ ]:
all_ge = {'Rock': 0, 'Hip Hop': 0, 'Pop': 0}

for g in df['Genre']:
    if str(g).find('Hip Hop') != -1:
        all_ge['Hip Hop'] = all_ge['Hip Hop'] + 1


for g in df['Genre']:
    if str(g).find('Rock') != -1:
        all_ge['Rock'] = all_ge['Rock'] + 1


for g in df['Genre']:
    if str(g).find('Pop') != -1:
        all_ge['Pop'] = all_ge['Pop'] + 1

print(all_ge)

In [ ]:
ge = {'Rock / pop': 0, 'Hip Hop / pop': 0, 'Rock / Hip Hop': 0}

for g in df['Genre']:
    if str(g).find('Hip Hop') != -1 and str(g).find('Rock') != -1:
        ge['Rock / Hip Hop'] = ge['Rock / Hip Hop'] + 1

for g in df['Genre']:
    if str(g).find('Rock') != -1 and str(g).find('Pop') != -1:
        ge['Rock / pop'] = ge['Rock / pop'] + 1

for g in df['Genre']:
    if str(g).find('Pop') != -1 and str(g).find('Hip Hop') != -1:
        ge['Hip Hop / pop'] = ge['Hip Hop / pop'] + 1

print(ge)

In [ ]:
most_HipHop = all_ge['Hip Hop'] - ge['Hip Hop / pop'] - ge['Rock / Hip Hop']
most_pop = all_ge['Pop'] - ge['Hip Hop / pop'] - ge['Rock / pop']
most_rock = all_ge['Rock'] - ge['Rock / Hip Hop'] - ge['Rock / pop']

hip_hop = all_ge['Hip Hop']
pop = all_ge['Pop']
rock = all_ge['Rock']

mf_HipHop = all_ge['Hip Hop']
mf_pop = all_ge['Pop'] - ge['Hip Hop / pop']
mf_rock = all_ge['Rock'] - ge['Rock / Hip Hop'] - ge['Rock / pop']

print(f'All    ==> Hip Hop: {hip_hop} - Pop: {pop} - Rock: {rock}')
print(f'Some   ==> Hip Hop: {most_HipHop} - Pop: {most_pop} - Rock: {most_rock}')
print(f'Modify ==> Hip Hop: {mf_HipHop} - Pop: {mf_pop} - Rock: {mf_rock}')

In [ ]:
94992 - 32262 - 874

In [40]:
df['Rock'] = df['Genre'].apply(func = (lambda genre: 1 if str(genre).find('Rock') != -1 else 0 ))
df['Hip Hop'] = df['Genre'].apply(func = (lambda genre: 1 if str(genre).find('Hip Hop') != -1 else 0 ))
df['Pop'] = df['Genre'].apply(func = (lambda genre: 1 if str(genre).find('Pop') != -1 else 0 ))

In [ ]:
df.head()

In [ ]:
df['Hip Hop'].sum(), df['Pop'].sum(), df['Rock'].sum()

In [43]:
LABEL_COLUMNS = ['Pop', 'Hip Hop', 'Rock']

In [ ]:
df[LABEL_COLUMNS].sum().sort_values().plot(kind='bar');

In [ ]:
song = str(df.iloc[0, 0])

print(song)

In [ ]:
' '.join(song.split())

In [ ]:
df.head()

In [ ]:
df[(df['Rock'] == 0) & (df['Hip Hop'] == 0) & (df['Pop'] == 0)]['Genre'].unique()

In [ ]:
dataset_df = df[(df['Rock'] == 1) | (df['Hip Hop'] == 1) | (df['Pop'] == 1)]

dataset_df.head()

In [ ]:
dataset_df.shape

In [ ]:
dataset_df['Hip Hop'].sum(), dataset_df['Pop'].sum(), dataset_df['Rock'].sum()

In [ ]:
dataset_df['Rock'] = dataset_df.apply(func = (lambda df: 0 if (df['Rock'] == 1 and df['Hip Hop'] == 1) else df['Rock']), axis=1)
dataset_df['Rock'] = dataset_df.apply(func = (lambda df: 0 if (df['Rock'] == 1 and df['Pop'] == 1) else df['Rock']), axis=1)

dataset_df['Pop'] = dataset_df.apply(func = (lambda df: 0 if  (df['Pop'] == 1 and df['Hip Hop'] == 1) else df['Pop']), axis=1)


print(f'All    ==> Hip Hop: {hip_hop} - Pop: {pop} - Rock: {rock}')
print(f'Some   ==> Hip Hop: {most_HipHop} - Pop: {most_pop} - Rock: {most_rock}')
print(f'Modify ==> Hip Hop: {mf_HipHop} - Pop: {mf_pop} - Rock: {mf_rock}')

dataset_df['Hip Hop'].sum(), dataset_df['Pop'].sum(), dataset_df['Rock'].sum()

In [ ]:
dataset_df.head()

In [ ]:
dataset_df.shape

### Tokenizer

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained(CHECKPOINT)

In [ ]:
token_lens = []

for txt in dataset_df.Lyric:
  tokens = tokenizer.encode(txt)
  token_lens.append(len(tokens))

In [ ]:
sns.distplot(token_lens, hist=True)
plt.xlim([0, 512]);
plt.xlabel('Token count');

In [ ]:
train_df, test_df = train_test_split(dataset_df, test_size=0.1, random_state=RANDOM_SEED)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=RANDOM_SEED)

train_df.shape, val_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df['Hip Hop'].unique()

In [ ]:
labes = dataset_df.iloc[0, 5:]
labes

In [ ]:
labes.values

### DataLoder

In [63]:
class LyricDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer: transformers.BertTokenizer, max_len: int = 512, include_row_text=False):
    self.data = data
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.include_row_text = include_row_text

  def __len__(self):
    return len(self.data)

  def __getitem__(self, item):
    data_row = self.data.iloc[item]

    lyric_text = str(data_row.Lyric)
    lyric_text = ' '.join(lyric_text.split())
    labels = data_row[self.data.columns.tolist()[5: ]]

    encoding = self.tokenizer.encode_plus(
        lyric_text,
        add_special_tokens=True,
        return_token_type_ids=False,
        truncation=True,
        return_attention_mask=True,
        max_length=self.max_len,
        padding='max_length',
        return_tensors='pt'
    )


    output =  {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.FloatTensor(labels)
    }

    if self.include_row_text:
        output['lyric_text'] = lyric_text

    return output

In [64]:
def create_data_loader(df, tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE, include_row_text=False, shuffle=False):
  ds = LyricDataset(
    data=df,
    tokenizer=tokenizer,
    max_len=max_len,
    include_row_text=include_row_text
  )

  return DataLoader(
    ds,
    shuffle=shuffle,
    batch_size=batch_size,
    num_workers=4,
  )

In [ ]:
train_data_loader = create_data_loader(df=train_df, tokenizer=tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = create_data_loader(df=val_df, tokenizer=tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE, shuffle=False)
test_data_loader = create_data_loader(df=test_df, tokenizer=tokenizer, max_len=MAX_LEN, batch_size=BATCH_SIZE, include_row_text=True, shuffle=False)

In [ ]:
data = next(iter(test_data_loader))
data.keys()

In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['labels'].shape)

### Model

In [69]:
class MusicGenreClassification(nn.Module):

  def __init__(self, n_classes: int = 3):
    super(MusicGenreClassification, self).__init__()
    self.bert = transformers.BertModel.from_pretrained(CHECKPOINT, return_dict=True)
    self.dropout = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask, return_dict=True):
    output = self.bert(
        input_ids,
        attention_mask=attention_mask
        )
    output = self.linear(output.pooler_output)

    return output

In [93]:
model = MusicGenreClassification(n_classes=3)
model = model.to(device)

### Parameter Efficient FineTuning

In [ ]:
# delta_model = opendelta.AdapterModel(model)
delta_model = opendelta.LoraModel(model)
delta_model.freeze_module(exclude=["deltas", "classifier"]) # leave the delta tuning modules and the newly initialized classification head tunable.
delta_model.log() # optional: to visualize how the `model` changes.

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

In [ ]:
F.softmax(model(input_ids, attention_mask), dim=1)

### Some Functions

In [77]:
def loss_fn(outputs, labels):
    return torch.nn.BCEWithLogitsLoss()(outputs, labels).to(device)

In [78]:
def coumpute_accuracy(predictions, labels, threshold=0.5):
  binray_predictions = (predictions > 0.5).double()
  pre_label_accuracy = (binray_predictions == labels).double().mean(dim=0)

  rock, hip_hop, pop = pre_label_accuracy
  average_accuracy = pre_label_accuracy.mean()

  return average_accuracy.double(), rock.double(), hip_hop.double(), pop.double()

In [ ]:
optimizer = transformers.AdamW(model.parameters(), lr=LEARN_RATE, correct_bias=False)

total_steps = len(train_data_loader) * EPOCHS

scheduler = transformers.get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

### Training

In [80]:
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler, n_examples, device):
    model = model.train()

    losses = []
    average_accuracy, rock, hip_hop, pop = (0, 0, 0, 0)

    for index, batch in enumerate(data_loader):
        input_ids = batch['input_ids'].to(device, dtype=torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.float)

        outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
       )

        loss = loss_fn(outputs, labels)

        average_accuracy_, rock_, hip_hop_, pop_ = coumpute_accuracy(torch.softmax(outputs, dim=1), labels)

        average_accuracy += average_accuracy_.item()
        rock += rock_.item()
        hip_hop += hip_hop_.item()
        pop += pop_.item()

        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()


    return {'Average Accuracy': average_accuracy, 'Rock': rock, 'Pop': pop, 'Hip Hop': hip_hop} , np.mean(losses)

In [81]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    average_accuracy, rock, hip_hop, pop = (0, 0, 0, 0)

    with torch.no_grad():
        for index, batch in enumerate(data_loader):
            input_ids = batch["input_ids"].to(device, dtype=torch.long)
            attention_mask = batch["attention_mask"].to(device, dtype=torch.long)
            labels = batch["labels"].to(device, dtype=torch.float)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            loss = loss_fn(outputs, labels)

            average_accuracy_, rock_, hip_hop_, pop_ = coumpute_accuracy(torch.softmax(outputs, dim=1), labels)

            average_accuracy += average_accuracy_.item()
            rock += rock_.item()
            hip_hop += hip_hop_.item()
            pop += pop_.item()

            losses.append(loss.item())

            return {'Average Accuracy': average_accuracy, 'Rock': rock, 'Pop': pop, 'Hip Hop': hip_hop} , np.mean(losses)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

    print('=' * 50)
    print(f'Epoch {epoch + 1} / {EPOCHS}')

    train_acc, train_loss = train_epoch(model=model, data_loader=train_data_loader, loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler, n_examples=len(train_df), device=device)
    print(f'Train loss {train_loss}, accuracy {train_acc}')

    val_acc, val_loss = eval_model(model=model, data_loader=val_data_loader, loss_fn=loss_fn, device=device, n_examples=len(val_df))
    print(f'Val loss {val_loss}, accuracy {val_acc}')

    print('=' * 50)

    history['train_acc'].append(train_acc['Average Accuracy'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc['Average Accuracy'])
    history['val_loss'].append(val_loss)


    if val_acc['Average Accuracy'] > best_accuracy:
        # torch.save(model.state_dict(), 'best_model_state.bin')
        delta_model.save_finetuned('bert_model_state')
        best_accuracy = val_acc['Average Accuracy']